# Day Map Challenge

Este notebook contiene el código para el desafío de mapas.

## Ejecutar en Google Colab

[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lalgonzales/scriptVarios/blob/main/notebooks/day_map_challenge.ipynb)

# Mapa 12 de noviembre

In [1]:
# Cargar modulos
import geopandas as gpd
import pandas as pd
import leafmap
from owslib.wfs import WebFeatureService

In [2]:
# Cargar las capas de datos desde el wfs del ICF

# Crear la conexion al servicio wfs
wfs_url = "http://geoserver.icf.gob.hn/icf/wfs"
wfs = WebFeatureService(url=wfs_url, version="1.1.0")

## Obtener listado de capas disponibles 
# capas_disponibles = list(wfs.contents)
# for capa in capas_disponibles:
#     print(capa)

# Crear la llamada a las capas de interés
capas_list = ["icf:Limite_HN", "icf:microcuencas_declaradas"]

# Crear los gdf de cada capa
gdf_capas_dic = {}
for capa in capas_list:
    capa_name = capa.split(":")[-1]
    response = wfs.getfeature(typename=capa, outputFormat="application/json")
    gdf = gpd.read_file(response)
    gdf_capas_dic[capa_name] = gdf

# print(gdf_capas_dic.keys())

In [3]:
# Preparar las microcuencas para el mapa
gdf_mc = gdf_capas_dic["microcuencas_declaradas"]
# gdf_mc.dtypes
# Eliminar los registros sin fecha
gdf_mc.dropna(subset=["fech_decl"], inplace=True)

# Obtener las fechas min y max
fecha_min = gdf_mc["fech_decl"].min()
fecha_max = gdf_mc["fech_decl"].max()

print(fecha_min, fecha_max)
# gdf_mc.fech_decl.unique

2008-02-15 00:00:00 2024-11-07 00:00:00


In [4]:
# Seleccionar las columnas de interes
gdf_mc = gdf_mc[["id", "anio_decla", "fech_decl", "area_ha", "geometry"]].copy()

# Ordenar los datos por fecha de declaración
gdf_mc = gdf_mc.sort_values(by="fech_decl").reset_index()

# Contar la cantidad de declaraciones por año
decl_by_year = gdf_mc.groupby(by=["anio_decla"])["anio_decla"].size()

# Eliminar 2008 ya que solo tiene 1 declaratoria y la siguientes inician en 2020
gdf_mc_gte_2020 = gdf_mc.query("anio_decla != 2008").copy()
gdf_mc_gte_2020.reset_index()
print(gdf_mc_gte_2020.shape[0], gdf_mc.shape[0])
gdf_mc_gte_2020.head()

275 276


,index,id,anio_decla,fech_decl,area_ha,geometry
1,593,microcuencas_declaradas.988,2020.0,2020-01-10,2816.24641,"MULTIPOLYGON (((533623.39 1670863.485, 533696...."
2,814,microcuencas_declaradas.1036,2020.0,2020-01-17,4.92363,"MULTIPOLYGON (((383512.207 1556106.209, 383478..."
3,409,microcuencas_declaradas.1033,2020.0,2020-01-29,27.25117,"MULTIPOLYGON (((346985.045 1615834.338, 346846..."
4,867,microcuencas_declaradas.1044,2020.0,2020-02-17,211.67794,"MULTIPOLYGON (((677947 1697706, 677961 1697614..."
5,15,microcuencas_declaradas.17,2020.0,2020-03-05,17.29032,"MULTIPOLYGON (((339003 1573145, 339147 1573126..."


In [5]:
# Crear df de area por año y el acumulado por año
df_acumulado = gdf_mc_gte_2020.groupby("anio_decla").agg(area_ha = ("area_ha", "sum")).reset_index()
df_acumulado["area_acc"] = df_acumulado["area_ha"].cumsum()
df_acumulado

,anio_decla,area_ha,area_acc
0,2020.0,4969.006433,4969.006433
1,2021.0,5688.960295,10657.966728
2,2022.0,9538.426864,20196.393592
3,2023.0,6710.120892,26906.514484
4,2024.0,3927.659917,30834.174401


In [18]:
Map = leafmap.Map()

# Definir los colores para cada año
legend_dict = {
    "2020": "#1f77b4", # color
    "2021": "#ff7f0e",
    "2022": "#2ca02c",
    "2023": "#d62728",
    "2024": "#9467bd",
}

style = {
    "position": "fixed",
    "z-index": "9999",
    "border": "2px solid grey",
    "background-color": "rgba(255, 255, 255, 0.8)",
    "border-radius": "10px",
    "padding": "5px",
    "font-size": "14px",
    "bottom": "20px",
    "right": "5px",
}

# Map.add_gdf(gdf_mc_gte_2020)
Map.add_legend(title="Area por anio", legend_dict=legend_dict, style=style, draggable=False)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
# Crear el mapa

# Crear la figura y los ejes
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# Ploteo básico de las microcuencas
gplt.polyplot(
    gdf_mc_gte_2020.explode(),
    ax=ax[0]
)
ax[0].set_title("Microcuencas Declaradas")

# Crear el gráfico de barras del área acumulada
sns.barplot(
    data=df_acumulado,
    x="anio_decla",
    y="area_acc",
    ax=ax[1],
    palette="viridis"
)
ax[1].set_title("Área Acumulada de Microcuencas Declaradas")
ax[1].set_xlabel("Año de Declaración")
ax[1].set_ylabel("Área Acumulada (ha)")

# Mostrar la figura
plt.tight_layout()
plt.show()

In [ ]:
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib_scalebar.scalebar import ScaleBar

# Asegurarse de que el GeoDataFrame esté en la proyección correcta
if gdf_mc_gte_2020.crs != "EPSG:4326":
    gdf_mc_gte_2020 = gdf_mc_gte_2020.to_crs(epsg=4326)

# Convertir la columna de fecha de declaración a datetime
gdf_mc_gte_2020["fech_decl"] = pd.to_datetime(gdf_mc_gte_2020["fech_decl"])

# Extraer el año de la fecha de declaración
gdf_mc_gte_2020["anio_declaracion"] = gdf_mc_gte_2020["fech_decl"].dt.year

# Agrupar por año de declaración y calcular el área acumulada
df_acumulado = gdf_mc_gte_2020.groupby("anio_declaracion")["area_ha"].sum().reset_index()
df_acumulado["area_acumulada"] = df_acumulado["area_ha"].cumsum()

# Crear la figura y los ejes
fig, ax = plt.subplots(1, 2, figsize=(20, 10), subplot_kw={"projection": ccrs.PlateCarree()})

# Ploteo básico de las microcuencas
gplt.polyplot(
    gdf_mc_gte_2020,
    ax=ax[0]
)
ax[0].set_title("Microcuencas Declaradas")

# Añadir una rosa náutica
ax[0].text(0.1, 0.1, "N", transform=ax[0].transAxes, ha="center", va="center", fontsize=12, fontweight="bold")
ax[0].arrow(0.1, 0.1, 0, 0.05, transform=ax[0].transAxes, color="black", head_width=0.02, head_length=0.02)

# Añadir una escala
scalebar = ScaleBar(1, location="lower right")
ax[0].add_artist(scalebar)

# Crear el gráfico de barras del área acumulada
sns.barplot(
    data=df_acumulado,
    x="anio_declaracion",
    y="area_acumulada",
    ax=ax[1],
    palette="viridis"
)
ax[1].set_title("Área Acumulada de Microcuencas Declaradas")
ax[1].set_xlabel("Año de Declaración")
ax[1].set_ylabel("Área Acumulada (ha)")

# Mostrar la figura
plt.tight_layout()
plt.show()
